## TVB TimeSeries Widget

#### This notebook is dedicated to showcasing the TimeSeries widget using TVB data

---

### Intial setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget

In [3]:
from tvbwidgets.api import TimeSeriesWidget
from IPython.core.display_functions import display

10-04-2022 08:59:16 - DEBUG - tvbwidgets - Package is not fully installed
/Users/lia.domide/WORK/TVB/tvb-widgets
10-04-2022 08:59:16 - DEBUG - tvbwidgets - Version read from the internal package.json file
10-04-2022 08:59:16 - INFO - tvbwidgets - Version: 0.2.0


/Library/Anaconda/anaconda3/envs/tvb-widgets/lib/python3.9/site-packages/tvb/datatypes/surfaces.py:63: UserWarning: Geodesic distance module is unavailable; some functionality for surfaces will be unavailable.
  warnings.warn(msg)


### Generate TVB data

 #### A --  This TS has only 1 state variable and 1 mode:

In [4]:
from tvbwidgets.tests.ts_generator import generate_ts_with_stimulus
tsr1 = generate_ts_with_stimulus()
tsr1

WARNING  File 'hemispheres' not found in ZIP.
   INFO  White noise configured with dt=0.5


,value
Dimensions,"('Time', 'State Variable', 'Region', 'Mode')"
Length,5.0
Region Mapping,None
Region Mapping Volume,None
Sample period,0.001
Source Connectivity,Connectivity gid: 2766f988-feec-42ad-89ae-e48650d2faa9
Time units,s
Time-series name,TimeSeriesRegion gid: 76e9a330-62a5-40cf-9e93-f06a95d05329
Time-series type,TimeSeriesRegion
"[min, median, max]","[-1.53398, 0.0317675, 3.65151]"


#### B -- Generate TVB TS data with more modes:

In [5]:
from tvbwidgets.tests.ts_generator import generate_ts_with_mode_and_sv
#tsr2 = generate_ts_with_mode_and_sv()
#tsr2

### Plot data using our TS widget

In [6]:
tsw = TimeSeriesWidget()
tsw.add_datatype(tsr1)

10-04-2022 08:59:27 - INFO - tvbwidgets.ui.ts_widget - Initializing TS Widget
10-04-2022 08:59:27 - DEBUG - tvbwidgets.ui.ts_widget - Adding TVB TS for display...
Creating RawArray with float64 data, n_channels=76, n_times=5000
    Range : 0 ... 4999 =      0.000 ...     4.999 secs
Ready.


In [7]:
display(tsw.get_widget())

Using matplotlib as 2D backend.
